# This is the reimplementation of the paper "EmoBERTa: Speaker-Aware Emotion Recognition in Conversation with RoBERTa" by Taewoon Kim and Piek Vossen, Vrije Universiteit Amsterdam.
# 
Link of the paper: https://arxiv.org/pdf/2108.12009.pdf

# 
Few parts of the code are copied directly from https://github.com/tae898/erc and are mentioned seperately in the code with the help  f comments.

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
import numpy as np
import json
import torch
from sklearn.metrics import f1_score
from tqdm import tqdm
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

2024-04-25 23:06:10.098226: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-25 23:06:11.063967: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# Opening JSON file
with open('Subtask_1_train.json') as json_file:
    data = json.load(json_file)

for i in range(len(data)):
    data[i].pop('conversation_ID')
    data[i].pop('emotion-cause_pairs')

new_data = []
for i in range(len(data)):
    new_data.append(data[i]['conversation'])

emo_dict = {"neutral":0, "joy":1, "surprise":2, "anger":3, "sadness":4, "disgust":5, "fear":6}

for i in range(len(new_data)):
    for j in range(len(new_data[i])):
        new_data[i][j].pop('utterance_ID')
        new_data[i][j].update({"utterance": new_data[i][j]["speaker"]+": "+new_data[i][j]["text"]})
        emotion = emo_dict[new_data[i][j]['emotion']]
        new_data[i][j].update({'emotion':emotion})
        new_data[i][j].pop('text')
        new_data[i][j].pop('speaker')

training_data = new_data[:1099].copy()
testing_data = new_data[1236:].copy()

In [10]:
class ErcDataset(torch.utils.data.Dataset):
    def __init__(self, data, num_past_utterances=0, num_future_utterances=0):
        """Initializer for emotion recognition in conversation text dataset."""
        self.data = data
        self.num_past_utterances = num_past_utterances
        self.num_future_utterances = num_future_utterances
        self.processed_data = self._create_input()

    def __len__(self):
        """Returns the length of the processed data."""
        return len(self.processed_data)

    def _create_input(self):
        """Creates inputs for RoBERTa."""
        tokenizer = AutoTokenizer.from_pretrained('/home/research/shaina mehta/RoBBERTa/Project/Tokenizer', use_fast=True)
        max_model_input_size = tokenizer.max_model_input_sizes['FacebookAI/roberta-base']
        inputs = []
        for dialog in self.data:
            num_truncated = 0
            for idx, utterance in enumerate(dialog):
                num_tokens = len(tokenizer(utterance["utterance"])["input_ids"])
                label = utterance["emotion"]
                indexes = [idx]
                indexes_past = [i for i in range(idx - 1, idx - self.num_past_utterances - 1, -1)]
                indexes_future = [i for i in range(idx + 1, idx + self.num_future_utterances + 1, 1)]
                offset = 0
                if len(indexes_past) < len(indexes_future):
                    indexes_past.extend([None] * (len(indexes_future) - len(indexes_past)))
                elif len(indexes_past) > len(indexes_future):
                    indexes_future.extend([None] * (len(indexes_past) - len(indexes_future)))
                for i, j in zip(indexes_past, indexes_future):
                    if i is not None and i >= 0:
                        indexes.insert(0, i)
                        offset += 1
                        if sum(num_tokens for idx_ in indexes) > max_model_input_size:
                            del indexes[0]
                            offset -= 1
                            num_truncated += 1
                            break
                    if j is not None and j < len(dialog):
                        indexes.append(j)
                        if sum(num_tokens for idx_ in indexes) > max_model_input_size:
                            del indexes[-1]
                            num_truncated += 1
                            break
                final_utterance = "</s></s>".join([dialog[idx_]["utterance"] for idx_ in indexes])
                input_ids_attention_mask = tokenizer(final_utterance, return_tensors="pt", padding="max_length", truncation=True, max_length=max_model_input_size)
                input_ = {
                    "input_ids": input_ids_attention_mask["input_ids"].squeeze(),
                    "attention_mask": input_ids_attention_mask["attention_mask"].squeeze(),
                    "label": label,
                }
                inputs.append(input_)
        return inputs

    def __getitem__(self, index):
        """Returns the processed data at the given index."""
        return self.processed_data[index]


In [11]:
model = AutoModelForSequenceClassification.from_pretrained('/home/research/shaina mehta/RoBBERTa/Project/Model Weights', num_labels=7)

In [12]:
training_dataset = ErcDataset(training_data,0,0)
validation_dataset = ErcDataset(testing_data,0,0)

In [13]:
import evaluate
f1 = evaluate.load("f1")
accuracy = evaluate.load('accuracy')

In [14]:
def compute_metrics(eval_predictions):
    preds = np.argmax(eval_predictions.predictions, axis=1)
    return {"f1 score": f1.compute(predictions=preds, references=eval_predictions.label_ids, average='macro'), 
            "accuracy": accuracy.compute(predictions=preds, references=eval_predictions.label_ids)}

In [15]:
tokenizer = AutoTokenizer.from_pretrained('/home/research/shaina mehta/RoBBERTa/Project/Tokenizer', use_fast=True)

In [16]:
# taken reference from https://github.com/tae898/erc
training_args = TrainingArguments(
    output_dir="/home/research/shaina mehta/RoBBERTa/Project/",
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=40,
    per_device_eval_batch_size=40,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=10,
    logging_strategy='epoch',
    metric_for_best_model="f1"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=training_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/media/data_dump/conda/miniconda3/lib/python3.11/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [17]:
trainer.evaluate()

{'eval_loss': 2.097917079925537,
 'eval_f1 score': {'f1': 0.43087254031226385},
 'eval_accuracy': {'accuracy': 0.5803389830508474},
 'eval_runtime': 10.8061,
 'eval_samples_per_second': 136.496,
 'eval_steps_per_second': 3.424}